In [1]:
import pandas as pd
import pickle
import seaborn as sns
import numpy as np
import itertools
from matplotlib import pyplot as plt
from datetime import datetime, timedelta

path = '../../../../../Desktop/Orthografietrainer2020/pickle/'

In [5]:
infile = open(path+'umfrage2020only.pkl','rb')
umfrage = pickle.load(infile)
infile.close()

#Deutschnote: bereinigen
umfrage['deutschnote'] = umfrage['deutschnote'].replace(['Es gab keine Noten', np.nan],0)
umfrage['deutschnote'] = umfrage['deutschnote'].astype('int')
# Erstschrift: deutsch=1, nicht deutsch=0
umfrage['ErstSchrift'] = umfrage['ErstSchrift'].replace([np.nan,'Russisch','Serbisch','Englisch','Italienisch','Vietnamesisch','Persisch','Kroatisch','Polnisch','Kurdisch','Arabisch','Griechisch','Spanisch','Niederl&auml;ndisch','T&uuml;rkisch','eine andere, und zwar...'],0)
umfrage['ErstSchrift'] = umfrage['ErstSchrift'].replace(['Deutsch'],1)
#Eigsprache: deutsch=1, nicht deutsch=0
umfrage['eigSprache'] = umfrage['eigSprache'].replace([np.nan,'Russisch','Serbisch','Englisch','Italienisch','Vietnamesisch','Persisch','Kroatisch','Polnisch','Kurdisch','Arabisch','Griechisch','Spanisch','Niederl&auml;ndisch','T&uuml;rkisch','eine andere, und zwar...'],0)
umfrage['eigSprache'] = umfrage['eigSprache'].replace(['Deutsch'],1)

# Sprache mutter: deutsch = 1, nicht deutsch = 0
umfrage['SpracheMutter'] = umfrage['SpracheMutter'].replace([np.nan,'Russisch','Serbisch','Englisch','Italienisch','Vietnamesisch','Persisch','Kroatisch','Polnisch','Kurdisch','Arabisch','Griechisch','Spanisch','Niederl&auml;ndisch','T&uuml;rkisch','eine andere, und zwar...'],0)
umfrage['SpracheMutter'] = umfrage['SpracheMutter'].replace(['Deutsch'],1)

# Sprache mutter: deutsch = 1, nicht deutsch = 0
umfrage['SpracheVater'] = umfrage['SpracheVater'].replace([np.nan,'Russisch','Serbisch','Englisch','Italienisch','Vietnamesisch','Persisch','Kroatisch','Polnisch','Kurdisch','Arabisch','Griechisch','Spanisch','Niederl&auml;ndisch','T&uuml;rkisch','eine andere, und zwar...'],0)
umfrage['SpracheVater'] = umfrage['SpracheVater'].replace(['Deutsch'],1)

#Abieltern: weißnicht = nan
umfrage['AbiEltern'] = umfrage['AbiEltern'].replace(['weissnicht'],np.nan)

#Buecher
umfrage['Buecher'] = umfrage['Buecher'].replace(['10'],0)
umfrage['Buecher'] = umfrage['Buecher'].replace(['50'],1)
umfrage['Buecher'] = umfrage['Buecher'].replace(['100'],2)
umfrage['Buecher'] = umfrage['Buecher'].replace(['200'],3)


li = ['freizeit_Fernsehen','fach_Spanisch', 'freizeit_Freunde',
       'fach_Sport', 'freizeit_Internet','freizeit_Kino', 'fach_Biologie', 
       'freizeit_Lesen', 'fach_Chemie', 'fach_keinLieblingsfach',
       'freizeit_Musikhoeren', 'fach_DS', 'fach_keineAngabe',
       'freizeit_Musikmachen', 'fach_Deutsch', 'freizeit_Basteln',
       'freizeit_Schreiben', 'fach_Englisch', 'freizeit_Computerspiele',
       'freizeit_Shopping', 'fach_Franzoesisch', 'freizeit_Familie','freizeit_Sport', 'fach_Geografie',
       'fach_Politik', 'freizeit_Tanzen', 'fach_Geschichte',
       'freizeit_Theater', 'fach_Handarbeit', 
       'fach_Informatik', 'freizeit_kaumFreizeit', 'fach_Kunst', 
       'fach_Musik', 'fach_Physik', 'freizeit_keineAngabe',
       'fach_Mathematik', 'fach_Sozialkunde', 'fach_Religion']

for x in li:
    umfrage[x] = umfrage[x].fillna(0)
    umfrage[x] = umfrage[x].astype('int')


#umfrage = umfrage.drop(columns=['klassenstufe', 'geschlecht', 'eigSpracheAndere','ErstSchriftAndere', 'SpracheMutterAndere','SpracheVaterAndere'])

b = [  'SchreibenGern', 'SchreibenLeicht','deutschnote',
       'eigSprache', 'ErstSchrift', 'SpracheMutter', 'LesenGern',
       'SpracheVater', 'LesenLeicht']#'AbiEltern',

for i in b:
    umfrage[i] = umfrage[i].astype('float32')
    umfrage[i] = umfrage[i].fillna(umfrage[i].mean())

umfrage = umfrage[['UserID','AbiEltern','Buecher','eigSprache']]

In [4]:
# import preprocessed data
infile = open('../04_predictionUserHistory/01_data/final_data_preprocessed.pkl','rb')
import_file = pickle.load(infile)
infile.close()
smaller_features=import_file[['UserID','Erstloesung','Erfolg','Schwierigkeit','Wochentag','ist_Schulzeit','MehrfachFalsch','Testposition__FT', 'Testposition__nt', 'Testposition__pruefung',
       'Testposition__training', 'Testposition__version', 'Testposition__vt',
       'Testposition__zt', 'beendet', 'Fehler', 'HA__HA', 'HA__Self', 'HA__nt',
       'HA__vt', 'HA__zt', 'Klassenstufe', 'Jahredabei',
       'Sex__m', 'Sex__w']]

userIds = list(umfrage['UserID'])
smaller_features = smaller_features[smaller_features['UserID'].isin(userIds)]

In [6]:
# import all sentences
infile = open(path+'saetze.pkl','rb')
saetze = pickle.load(infile)
infile.close()
saetze = saetze[['satzID','Schwierigkeit','Art','AufgabenID']]

#Art: only GK
options = ['GK'] 
saetze = saetze[saetze['Art'].isin(options)] 

#Aufgaben ID : all except 0 (0 is kompetenztests) 
options = ['0'] 
saetze = saetze[~saetze['AufgabenID'].isin(options)] 

In [7]:
# create empty df with null values
# rows: all userIDs from preprocessed and balanced dataset
# columns: all satzIDs

userIDs = smaller_features[['UserID']]
userIDs = userIDs.drop_duplicates()
index_col = userIDs['UserID'].tolist()
satzList = saetze["satzID"].tolist()
df = pd.DataFrame(0, index =index_col,columns =satzList)

In [8]:
# import xmlsaetze and xmlsaetze archiv
# important: do not only use xmlsaetze from preprocessed sentence! although we only use a balanced subset, we need all historical data

infile = open(path+'xmlsaetze.pkl','rb')
xmlsaetze = pickle.load(infile)
infile.close()
xmlsaetze = xmlsaetze[['ID','UserID','UebungsID','SatzID','Erfolg','Datum']]

infile = open(path+'xmlsaetze_archiv.pkl','rb')
xmlsaetze_archiv = pickle.load(infile)
infile.close()
xmlsaetze_archiv = xmlsaetze_archiv[['ID','UserID','UebungsID','SatzID','Erfolg','Datum']]

xmlsaetze = xmlsaetze.append(xmlsaetze_archiv)
xmlsaetze_small = xmlsaetze[['UserID','SatzID','Erfolg','Datum']]

# filter satzID: only GK and not kompetenztests
options = saetze['satzID'].tolist()
xmlsaetze_small = xmlsaetze_small[xmlsaetze_small['SatzID'].isin(options)] 

# filter unserID: only User which appear in preprocessed dataset
options = index_col
xmlsaetze_userid = xmlsaetze_small[xmlsaetze_small['UserID'].isin(options)] 


In [9]:
# add column last_login
# im live system ist das nicht notwendig, da hier der aktuelle Tag genommen wird
# beim Training der Daten muss es aber schon genommen werden, weil sonst nur Okt, Nov, Dez zum trainineren genutzt wird, obwohl User vllt im Herbst gar nicht online waren
last_login = xmlsaetze_userid[['UserID','Datum']].groupby(['UserID']).max()
last_login = pd.DataFrame(last_login.reset_index())
last_login = last_login.rename(columns={"Datum": "last_login"})

# merge last login zu datafrage
xmlsaetze_userid_last_login= xmlsaetze_userid.merge(last_login, on='UserID', how='left')

In [10]:
#iterate trough dataframe row-wise

for i in range(xmlsaetze_userid_last_login.shape[0]): #iterate over rows
    user_cell = xmlsaetze_userid_last_login.iloc[i,0]
    satz_cell = xmlsaetze_userid_last_login.iloc[i,1]
    erfolg_cell = xmlsaetze_userid_last_login.iloc[i,2]
    datum_cell = xmlsaetze_userid_last_login.iloc[i,3]
    last_login_cell = xmlsaetze_userid_last_login.iloc[i,4]

    dateOfExercise = datetime.strptime(str(datum_cell), '%Y-%m-%d %H:%M:%S').date()
    dateOfLastLogin = datetime.strptime(str(last_login_cell), '%Y-%m-%d %H:%M:%S').date()
    acceptedDate = dateOfLastLogin + pd.DateOffset(months=-3) # accepted date calculates the date of the last login minus 3 months

    #check if sentence and user is in df
    if satz_cell in df.columns:
        if user_cell in df.index:
            if(dateOfExercise > acceptedDate):
                # set new cell value
                if(erfolg_cell == 1):
                    df.loc[user_cell,satz_cell] = 1
                elif (erfolg_cell ==0):
                    df.loc[user_cell,satz_cell] = -1          

In [11]:
historical_join = df.reset_index()
historical_join = historical_join.rename(columns={"index": "UserID"})
historical_join.to_pickle('historicalJoin_3months.pkl')

dataset = pd.merge(smaller_features, historical_join, how='left', on='UserID')
dataset.to_pickle('smallSampleSet_3months.pkl')

In [2]:
infile = open('smallSampleSet_3months.pkl','rb')
dataset = pickle.load(infile)
infile.close()

In [3]:
dataset

,UserID,Erstloesung,Erfolg,Schwierigkeit,Wochentag,ist_Schulzeit,MehrfachFalsch,Testposition__FT,Testposition__nt,Testposition__pruefung,...,9992992,9992993,9992994,9992995,9992996,9992997,9992998,9992999,9993000,9993001
0,509878,1,0,-1.207390,2,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,509878,0,1,-1.207390,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,509878,0,1,-1.207390,2,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,509881,0,1,-1.207390,6,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,509881,0,1,-1.207390,6,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886824,721266,1,1,0.876031,5,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
886825,658757,1,1,2.017740,5,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
886826,532914,1,1,2.017740,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
886827,432667,1,0,2.017740,2,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
dataset = dataset[dataset['Erfolg'] != -1]
dataset

,UserID,Erstloesung,Erfolg,Schwierigkeit,Wochentag,ist_Schulzeit,MehrfachFalsch,Testposition__FT,Testposition__nt,Testposition__pruefung,...,9992992,9992993,9992994,9992995,9992996,9992997,9992998,9992999,9993000,9993001
0,509878,1,0,-1.207390,2,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,509878,0,1,-1.207390,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,509878,0,1,-1.207390,2,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,509881,0,1,-1.207390,6,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,509881,0,1,-1.207390,6,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886824,721266,1,1,0.876031,5,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
886825,658757,1,1,2.017740,5,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
886826,532914,1,1,2.017740,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
886827,432667,1,0,2.017740,2,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
dataset

In [6]:
df = pd.merge(dataset, umfrage, on='UserID')
df.to_pickle('preprocessed_umfrage.pkl')

In [ ]:
infile = open('preprocessed_umfrage.pkl','rb')
df = pickle.load(infile)
infile.close()

In [7]:
################### SUBGROUPING

# Abi eltern
df.AbiEltern = df.AbiEltern.astype('float')
df_abi = df[df.AbiEltern > 0]
df_keinAbi = df[df.AbiEltern ==0]
df_weissnicht = df[df.AbiEltern.isnull()]

print(len(df_abi.UserID.unique()))
print(len(df_keinAbi.UserID.unique()))
print(len(df_weissnicht.UserID.unique()))

#Gender
df_boys = df[df.Sex__m == 1]
df_girls = df[df.Sex__w == 1]
print(len(df_boys.UserID.unique()))
print(len(df_girls.UserID.unique()))

# Migration
df_deutsch = df[df.eigSprache == 1]
df_migration = df[df.eigSprache == 0]

print(len(df_deutsch.UserID.unique()))
print(len(df_migration.UserID.unique()))

# Anzahl Bücher
df['Buecher'] = df['Buecher'].replace(['10'],0)
df['Buecher'] = df['Buecher'].replace(['200'],1)
df_buch0 = df[df.Buecher == 0.0]
df_buch1 = df[df.Buecher == 1]

print(len(df_buch0.UserID.unique()))
print(len(df_buch1.UserID.unique()))

1556
641
1471
1760
1800
2163
1505
230
535


In [8]:
df_abi.to_pickle('double_df_abi.pkl')
df_keinAbi.to_pickle('double_df_keinAbi.pkl')
df_weissnicht.to_pickle('double_df_weissnicht.pkl')
df_boys.to_pickle('double_df_boys.pkl')
df_girls.to_pickle('double_df_girls.pkl')
df_deutsch.to_pickle('double_df_deutsch.pkl')
df_migration.to_pickle('double_df_migration.pkl')
df_buch0.to_pickle('double_df_buch0.pkl')
df_buch1.to_pickle('double_df_buch1.pkl')